In [36]:
# import modules
import requests
import json
import time
import os
from dotenv import load_dotenv # Need to use this in anaconda prompt: pip3 install python-dotenv

# load .env file
load_dotenv("APIKey.env")

# get api key from .env file
key = os.environ.get("key")

# The specific Model id. Insert it here.
model_id = "1cc96b76-92f7-4253-9092-bfb5d5de7d52"

# If the code is to be used elsewhere, insert the API key here instead of the variable.
API_KEY = key

HEADERS = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {API_KEY}"
}

In [37]:
def get_model_version(model_id, version_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/versions/{version_id}"
    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    version_id = data["id"]
    status = data["status"]

    print(f"Version ID: {version_id}. Status: {status}")

    return version_id, status

In [38]:
def generate_image(model_id, prompt):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences"

    payload = {
        "prompt": prompt,
        "steps": 50,
        "width": 512,
        "height": 512,
        "numberOfImages": 2,
        "seed": 4523184
    }

    response = requests.post(url, json=payload, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    status = data["status"]

    print(f"Inference ID: {inference_id}. Status: {status}")

    return inference_id, status

In [39]:
def get_inference_job(model_id, inference_id):
    url = f"https://api.tryleap.ai/api/v1/images/models/{model_id}/inferences/{inference_id}"

    response = requests.get(url, headers=HEADERS)
    data = json.loads(response.text)

    inference_id = data["id"]
    state = data["state"]
    image = None

    if len(data["images"]):
        image = data["images"][0]["uri"]

    print(f"Inference ID: {inference_id}. State: {state}")

    return inference_id, state, image

In [41]:
# Now that we have a fine-tuned version of a model, we can
# generate images using it. Notice how I'm using '@me' to 
# indicate I want pictures similar to the ones we upload to 

# Prompt text. Insert text here. Remember to use the @
promptText = "A photo of @Andreas ultrarealistic, half of the face in shadow, looking serious"

inference_id, status = generate_image(
    model_id, 
    prompt = promptText
)
while status != "finished":
    time.sleep(10)
    inference_id, status, image = get_inference_job(model_id, inference_id)

print(image)

Inference ID: 4cf583e6-54a6-4a6f-972a-180cb29c40e8. Status: queued
Inference ID: 4cf583e6-54a6-4a6f-972a-180cb29c40e8. State: queued
Inference ID: 4cf583e6-54a6-4a6f-972a-180cb29c40e8. State: queued
Inference ID: 4cf583e6-54a6-4a6f-972a-180cb29c40e8. State: finished
https://dreamtrain.s3.us-west-2.amazonaws.com/image-gen-4cf583e6-54a6-4a6f-972a-180cb29c40e8/generated_images/1.png
